In [1]:
import datetime as dt
from lxml import html
from bs4 import BeautifulSoup
import urllib
import sys
import requests
import operator
from datetime import timedelta
import csv
import pandas as pd
import numpy as np
import math

In [2]:
year1415 = pd.read_csv('GamesData/season1415.csv', parse_dates=['Date'])
year1516 = pd.read_csv('GamesData/season1516.csv', parse_dates=['Date'])
year1617 = pd.read_csv('GamesData/season1617.csv', parse_dates=['Date'])
year1718 = pd.read_csv('GamesData/season1718.csv', parse_dates=['Date'])
year1819 = pd.read_csv('GamesData/season1819.csv', parse_dates=['Date'])
year1920 = pd.read_csv('GamesData/season1920.csv', parse_dates=['Date'])

In [3]:
frames = [year1617,year1718,year1819,year1920]

In [4]:
allyears = pd.concat(frames)
allyears = allyears.drop_duplicates(subset=['Date', 'Home'],keep='last')
full = allyears[['Home','Away','WinnerHome','KenPomVal_Away','BPIval_Away','SRS_Away','KenPomVal_Home','BPIval_Home','SRS_Home']]


In [5]:
allyears

,Date,Win_Score,Lose_Score,Home,Away,WinnerHome,KenPomRank_Away,KenPomVal_Away,BPIrank_Away,BPIval_Away,...,d_eff_Home,ppg_Home,papg_Home,Pace_Home,Three_Par_Home,True_Shoot_Home,Tov_Home,Opp_Three_Par_Home,Opp_True_Shoot_Home,Opp_Tov_Home
0,2016-11-11,94,49,Duke,Marist,1,312.0,-13.63,305.0,-8.0,...,91.17,80.8,70.2,69.9,0.383,0.588,14.2,0.288,0.510,14.9
1,2016-11-11,87,64,Kentucky,Stephen F. Austin,1,233.0,-6.19,262.0,-5.2,...,86.87,84.9,71.5,74.3,0.316,0.564,13.4,0.333,0.507,16.9
2,2016-11-11,103,99,Kansas,Indiana,0,44.0,14.49,32.0,10.6,...,89.65,83.2,71.9,71.4,0.359,0.583,15.1,0.342,0.515,16.1
3,2016-11-11,88,48,Villanova,Lafayette,1,330.0,-16.79,330.0,-10.9,...,87.45,77.2,62.7,65.6,0.433,0.612,15.0,0.400,0.506,17.6
4,2016-11-11,91,77,Oregon,Army,1,230.0,-6.04,253.0,-4.7,...,88.20,78.9,65.8,68.8,0.388,0.582,14.7,0.378,0.494,16.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,2020-02-20,90,60,Coastal Carolina,Troy,1,285.0,-10.01,305.0,-7.6,...,102.94,78.2,76.0,74.2,0.356,0.542,18.2,0.447,0.544,16.9
4234,2020-02-20,69,58,Utah,UCLA,0,92.0,7.80,106.0,3.7,...,95.61,70.6,69.8,68.1,0.386,0.546,17.3,0.352,0.527,15.9
4235,2020-02-20,63,54,Stony Brook,Vermont,0,59.0,11.82,41.0,9.3,...,97.10,71.2,66.7,70.5,0.427,0.522,17.7,0.419,0.491,16.5
4236,2020-02-20,61,51,Towson,William & Mary,0,185.0,-1.44,189.0,-0.9,...,100.42,70.0,65.9,66.3,0.335,0.528,15.3,0.424,0.512,15.0


In [6]:
full = full.dropna()

In [7]:
full

,Home,Away,WinnerHome,KenPomVal_Away,BPIval_Away,SRS_Away,KenPomVal_Home,BPIval_Home,SRS_Home
0,Duke,Marist,1,-13.63,-8.0,-12.33,24.17,17.7,21.56
1,Kentucky,Stephen F. Austin,1,-6.19,-5.2,-7.01,27.72,17.5,23.88
2,Kansas,Indiana,0,14.49,10.6,14.63,27.45,18.4,23.53
3,Villanova,Lafayette,1,-16.79,-10.9,-14.68,29.88,20.3,23.80
4,Oregon,Army,1,-6.04,-4.7,-5.84,25.02,17.4,20.80
...,...,...,...,...,...,...,...,...,...
4233,Coastal Carolina,Troy,1,-10.01,-7.6,-8.84,-1.83,-1.3,-0.64
4234,Utah,UCLA,0,7.80,3.7,8.46,4.36,2.3,7.91
4235,Stony Brook,Vermont,0,11.82,9.3,6.82,1.34,1.9,-0.62
4236,Towson,William & Mary,0,-1.44,-0.9,-2.09,0.15,0.9,-0.22


In [8]:
X = full.iloc[:,[3,4,5,6,7,8]]
y = full.iloc[:,2]

In [9]:
y

0       1
1       1
2       0
3       1
4       1
       ..
4233    1
4234    0
4235    0
4236    0
4237    1
Name: WinnerHome, Length: 20171, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=14)

In [11]:
LR = LogisticRegression().fit(X_train, y_train)

In [12]:
LR.score(X_test, y_test)

0.770158625247852

In [13]:
LR.coef_ , LR.intercept_, LR.n_iter_

(array([[-0.14904683,  0.02408791, -0.00285878,  0.14617775, -0.05714497,
          0.03016929]]), array([0.53923323]), array([35]))

In [14]:
from sklearn import metrics
y_pred = LR.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[1278,  887],
       [ 504, 3383]], dtype=int64)

In [15]:
def conf(game):
    arr = LR.coef_
    print("Coefficients:",arr)
    print("Team Stats:",game)
    coefmult = arr*game
    print("Multiplied:",coefmult)
    comb_coef = coefmult.sum()+LR.intercept_
    print("Sum of multiplied:",comb_coef)
    es = math.exp(comb_coef)
    print("E to the sum of multiplied:",es)
    confidence = es / (1+es)
    print("conf:", confidence)

In [16]:
LSUatArk = [10.8,6.3,10.51,31.01,18.4,25.79]
predict = [LSUatArk]
res = LR.predict_proba(predict)
confidence = conf(LSUatArk)
print()
print("Prob from Model",res)

Coefficients: [[-0.14904683  0.02408791 -0.00285878  0.14617775 -0.05714497  0.03016929]]
Team Stats: [10.8, 6.3, 10.51, 31.01, 18.4, 25.79]
Multiplied: [[-1.60970578  0.15175382 -0.03004582  4.53297218 -1.05146737  0.77806606]]
Sum of multiplied: [3.31080633]
E to the sum of multiplied: 27.407215701165324
conf: 0.9647976763889966

Prob from Model [[0.03520232 0.96479768]]
